
### **Template Agent: `agent.py`**

In [1]:
cat agent.py

from autogen_core import MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
import messages
import random


class Agent(RoutedAgent):

    # Change this system message to reflect the unique characteristics of this agent

    system_message = """
    You are a creative entrepreneur. Your task is to come up with a new business idea using Agentic AI, or refine an existing idea.
    Your personal interests are in these sectors: Healthcare, Education.
    You are drawn to ideas that involve disruption.
    You are less interested in ideas that are purely automation.
    You are optimistic, adventurous and have risk appetite. You are imaginative - sometimes too much so.
    Your weaknesses: you're not patient, and can be impulsive.
    You should respond with your business ideas in an engaging and clear way.
    """

    CHANCES_THA

---

**Purpose**

* `agent.py` is our template.
* This file is given to another agent, asking it to use this as its model/example.
* The idea is to **take this as a template** and make other agents like this.
* It's simply a **prototype** to be cloned and varied — **by an agent, not by us**.

**Imports**

* The file does some necessary imports.

**System Message**

* Sets a system message describing the agent’s personality and mission:

  > “You are a creative entrepreneur. Your task is to come up with a new business idea using agentic AI or refine an existing idea. Your personal interests are in these sectors…”
* Drawn to disruption, less interested in pure automation, optimistic, adventurous, risk-taking, imaginative, impatient, sometimes impulsive.
* Should respond in an engaging and clear way.
* **Comment at the top:**
  “Change this system message to reflect the unique characteristics of this agent.”

**Behavior Constants**

* A constant:
  `chances that I bounce an idea off another is 0.5`
* **Comment:**
  “You can also change the code to make the behavior different, but be careful to keep the method signatures the same.”

**Initialization (`__init__`)**

* Sets `gpt40mini` as the LLM with a temperature of 0.7 (for more randomness).
* Creates a delegate assistant using this model client and the defined system message.

**Main Message Handler (`handleMyMessage` / `handleMessage`)**

* Decorated as a message handler.
* Takes a `message` (which uses the `messages` object, now separated into its own package for minimal code/reduced mistakes).
* Signature: takes a `messages.message`, returns a `messages.message`.
* Prints when it receives a message, including its type (like the agent’s name).

**Core Logic:**

1. Makes a text message from the input.
2. Sends this message to the underlying LLM (with the system prompt in context).
3. Waits for and receives the LLM response, treating it as a new business idea.
4. Picks a random number (`random.random()` between 0 and 1).

   * If below 0.5, will *bounce* the idea to another agent for refinement (using a utility function `find_recipient`).
   * Sends a message: “Here is my business idea. This might not be your specialty, but please refine it and make this business idea better.”
   * Calls `self.sendMessage()` (leveraging the runtime) to send to the randomly picked recipient.
   * Returns either its original idea, or a refined version, depending on the random outcome.

**Summary**

* **This agent is a template**:
  It can be cloned by other agents to create variants, each possibly with different personalities or behaviors, but with the same basic logic and message-handling capability.
* You can see how agents receive and send messages, sometimes collaborating (with some probability).
* It’s designed to allow dynamic, creative “agent creation” and interaction, with plenty of room for experimentation in multi-agent systems.


### `messages.py` 

In [2]:
cat messages.py

from dataclasses import dataclass
from autogen_core import AgentId
import glob
import os


import random

@dataclass
class Message:
    content: str


def find_recipient() -> AgentId:
    try:
        agent_files = glob.glob("agent*.py")
        agent_names = [os.path.splitext(file)[0] for file in agent_files]
        agent_names.remove("agent")
        agent_name = random.choice(agent_names)
        print(f"Selecting agent for refinement: {agent_name}")
        return AgentId(agent_name, "default")
    except Exception as e:
        print(f"Exception finding recipient: {e}")
        return AgentId("agent1", "default")


---
**Purpose**

* `messages.py` defines the core messaging logic and structures for agent communication.

**Key Points:**

* **Data Class:** Contains a familiar data class for the message object.
* **findRecipient:**

  * Function to select another agent to communicate with.
  * Very “hacky” logic:

    * As agents (clones) are created (agent1, agent2, agent3…), the function scans the directory for existing agents.
    * It randomly selects one of the available agent files.
    * There’s a chance the agent might talk to itself (by being selected as its own recipient).
    * This could be improved to avoid self-messaging, but is left as-is for now.


### `creator.py`

In [3]:
cat creator.py

from autogen_core import MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
import messages
from autogen_core import TRACE_LOGGER_NAME
import importlib
import logging
from autogen_core import AgentId

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(TRACE_LOGGER_NAME)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)


class Creator(RoutedAgent):

    # Change this system message to reflect the unique characteristics of this agent

    system_message = """
    You are an Agent that is able to create new AI Agents.
    You receive a template in the form of Python code that creates an Agent using Autogen Core and Autogen Agentchat.
    You should use this template to create a new Agent with a unique system message that is different from the template,
    and reflects their unique char

---

* `creator.py` is the AGENT that *creates* and *spawns* new agents dynamically.

**Key Points:**

* **System Message:**

  * The agent receives instructions to generate new agent code from a template,
    using AutogenCore and AutogenAgentChat.
  * Flexibility to change the agent’s personality or system prompt.
  * The only requirements: the new class must be named `agent`, must inherit from `rooted agent`, and must have an `__init__` method.
  * Output must be only Python code.

* **How Creation Works:**

  * The creator agent reads `agent.py` as the template.
  * It asks an LLM to mutate the code, giving each agent a unique system message, and possibly other variations.
  * Each new agent is saved as `agent1.py`, `agent2.py`, etc.
  * Uses `importlib` to dynamically import the new agent module it just wrote.
  * Registers the newly imported agent with the runtime, using a lambda/factory for instantiation.
  * Declares the new agent “live”:

    * Not just “deployed” in the software sense, but *running* and ready to receive messages.
  * Immediately sends the new agent a message, triggering it to generate and possibly refine a business idea.

* **Agent Interaction:**

  * Agents can message each other, or sometimes themselves, for idea refinement.
  * Demonstrates true agent-to-agent messaging and autonomy.
  * Built on asynchronous Python for efficiency and concurrency.

**What Makes This Architecture Special?**

* **Dynamically extensible:** Agents *generate code*, spawn new agents, and register them at runtime.
* **Inter-process communication:**

  * No need to worry about networking or plumbing—the platform takes care of routing messages, even across runtimes.
* **Creativity and risk:**

  * The agents are free to mutate their own logic and personality, within some guardrails.
* **Async Python:**

  * The system is designed to scale and handle concurrent agent activity.

**What Next?**

* I can:

  * **Explain the exact logic in your files** (line-by-line or by function/class).
  * **Suggest improvements** (e.g., to avoid self-messaging).
  * **Visualize** the flow with diagrams.
  * **Help you adapt or extend** this for a specific project.

**What would you like to do next?**
For example:

* “Show me a code walkthrough for messages.py”
* “Suggest improvements for the findRecipient logic”
* “Explain the dynamic import in creator.py”
* “Show how to make it avoid self-messaging”
* Or, “Run an example interaction with agent creation and messaging (pseudo-code)”

Let me know!

## **`World.py`: The Orchestrator Script**

In [5]:
cat world.py

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
from agent import Agent
from creator import Creator
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime
from autogen_core import AgentId
import messages
import asyncio
# Load .env file
from dotenv import load_dotenv
import os
load_dotenv(os.path.join(os.path.dirname(__file__), '../..', '.env'))

HOW_MANY_AGENTS = 20

async def create_and_message(worker, creator_id, i: int):
    try:
        result = await worker.send_message(messages.Message(content=f"agent{i}.py"), creator_id)
        with open(f"idea{i}.md", "w") as f:
            f.write(result.content)
    except Exception as e:
        print(f"Failed to run worker {i} due to exception: {e}")

async def main():
    host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
    host.start() 
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    result = await Creator.register(worker, "Creator", lambda: Creator("Crea



---

We're nearly there. I've just got one more thing to show you, which is `world.py`, which is quite short, which is the overall thing.

**Role of world.py**

`creator.py`—you have to send it, it's an agent, and you have to send Creator a message, like agent1, agent2, agent3, and it will create them. So this is where it all comes together.

`world.py` is **not an agent**. This is just a Python script. Everything else you've seen has been agents. This is just a Python script, but it uses async Python in quite an interesting way.

**The createAndMessage Coroutine**

So there is this async function, this coroutine, `createAndMessage`, which takes a worker and a creator ID, and an i, which is going to be the agent number. We have 1, 2, 3, 4, 5.
This "how many agents up here" is a pretty important one to look at. This is how many of these it's going to kick off at the same time.

**Cost and Model Risk**

And the other thing that's unsafe about this, of course, is cost. This is going to hit the real APIs, and for me, it's cost like a couple of cents when I've done this for 20 agents, which is pretty cheap all things considered.
And obviously, if you do it for like three agents, it costs nothing. But there's a risk there that we are allowing—in theory—the agent creator could change the model requirements. It could decide to use an expensive model instead of GPT-4 over any.
And so that's just something to watch for. It's very unlikely, but it's not like it's not impossible.

**Async Method: Creating and Messaging Agents**

Anyways, we've got 20 agents here, so we've got this async method, creator, and endMessage. It takes this worker and it sends a message to the agent number whatever dot dot pi, and then it writes the result to idea whatever dot marker. And that's all this thing here does.

**Main Async Function: Event Loop and Parallelism**

Okay, so now we go to our main async method and our main async care function, our covert team. So it creates a host, a gRPC worker agent runtime host, as we did in the notebook before. It starts the host, it creates a worker, and it starts the worker.

It then registers the creator. So it registers the creator from creator.py that was imported here. So that will launch a creator. And this is its ID, creator default.

**Parallel Execution with asyncio.gather**

Then this is the async thing. So I don't want to have a loop and call this creating worker after worker, sorry, creating agent after agent. Because then it will be serial. It will happen one after another.
If I do await and then I call this coroutine, then we'll be waiting there. It will create one file, it will send it a message, back will come the answer, maybe it will send it somewhere else, and then it will go on to the next. And we'll be sitting here waiting, and it won't be exciting.

But what you can do with coroutines, as you hopefully remember from when we briefly talked about this, is that you can get a whole stash of coroutines together. So I haven't got the word await here, which you need if this is actually going to run.
I'm just gathering a whole list of coroutines. And then I await them using `asyncIO.gather`. And `asyncIO.gather` runs them all in parallel.

**Difference from Multithreading**

And as hopefully you remember, it's not like multithreading. They're not going to be actually running on different threads, like with the CPU chopping between them.
But rather, they're running in the event loop. That's how asyncIO Python works. They run an event loop so that every time it's waiting on OpenAI, which means it's waiting on a network connection, another one can be running.
And that means they all get to run at the same time. As long as I stay within my rate limits with OpenAI, we'll be able to run plenty in parallel.

**Completion and Exception Handling**

And so it does all of that, and then at the end it stops, and any exceptions, it will print them. And that's it.

**How to Run async Python**

This is, by the way, how you run asyncPython from the command line or from a Python module. Your main file needs to do this, which brings, which initiates an `asyncIO` event loop and then runs your async method, your coroutine.

So that is how it hangs together.

**Project Summary**

And I do believe this is it. I don't think I've got anything else to show you. I think you've seen everything.
You've seen agent, which is the prototype, the thing that gets cloned. Creator, which does the cloning agent by agent. It's called one by one. And world.py is this.
It's not an agent. It's just Python code, which will orchestrate, which will launch the whole process running. Okay.



### Run

```sh
(agents_env) ➜  week5_autogen git:(main) ✗ uv run world.py
warning: `VIRTUAL_ENV=/Users/alex/Desktop/00_projects/AI_agents/my_agents/agents_env` does not match the project environment path `/Users/alex/Desktop/00_projects/AI_agents/my_agents/.venv` and will be ignored; use `--active` to target the active environment instead
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1754304522.386281  234843 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.410519  234844 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.429256  234903 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.474222  234844 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.498081  234843 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.503124  234936 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.529354  234933 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.543978  234934 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.553676  234903 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.558520  234935 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1754304522.574400  234937 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
** Creator has created python code for agent agent12 - about to register with Runtime
** Agent agent12 is live
INFO:autogen_core.trace:** Agent agent12 is live
agent12: Received message
** Creator has created python code for agent agent14 - about to register with Runtime
** Agent agent14 is live
INFO:autogen_core.trace:** Agent agent14 is live
agent14: Received message
** Creator has created python code for agent agent11 - about to register with Runtime
** Creator has created python code for agent agent1 - about to register with Runtime
** Agent agent11 is live
INFO:autogen_core.trace:** Agent agent11 is live
** Agent agent1 is live
INFO:autogen_core.trace:** Agent agent1 is live
agent11: Received message
agent1: Received message
** Creator has created python code for agent agent20 - about to register with Runtime
** Agent agent20 is live
INFO:autogen_core.trace:** Agent agent20 is live
agent20: Received message
** Creator has created python code for agent agent19 - about to register with Runtime
** Agent agent19 is live
INFO:autogen_core.trace:** Agent agent19 is live
agent19: Received message
** Creator has created python code for agent agent13 - about to register with Runtime
** Agent agent13 is live
INFO:autogen_core.trace:** Agent agent13 is live
agent13: Received message
** Creator has created python code for agent agent10 - about to register with Runtime
** Creator has created python code for agent agent17 - about to register with Runtime
** Agent agent10 is live
INFO:autogen_core.trace:** Agent agent10 is live
** Agent agent17 is live
INFO:autogen_core.trace:** Agent agent17 is live
agent10: Received message
agent17: Received message
** Creator has created python code for agent agent7 - about to register with Runtime
** Agent agent7 is live
INFO:autogen_core.trace:** Agent agent7 is live
agent7: Received message
** Creator has created python code for agent agent18 - about to register with Runtime
** Creator has created python code for agent agent9 - about to register with Runtime
** Agent agent18 is live
INFO:autogen_core.trace:** Agent agent18 is live
** Agent agent9 is live
INFO:autogen_core.trace:** Agent agent9 is live
agent18: Received message
agent9: Received message
** Creator has created python code for agent agent2 - about to register with Runtime
** Agent agent2 is live
INFO:autogen_core.trace:** Agent agent2 is live
agent2: Received message
** Creator has created python code for agent agent3 - about to register with Runtime
** Agent agent3 is live
INFO:autogen_core.trace:** Agent agent3 is live
agent3: Received message
** Creator has created python code for agent agent8 - about to register with Runtime
** Agent agent8 is live
INFO:autogen_core.trace:** Agent agent8 is live
agent8: Received message
** Creator has created python code for agent agent6 - about to register with Runtime
** Agent agent6 is live
INFO:autogen_core.trace:** Agent agent6 is live
agent6: Received message
** Creator has created python code for agent agent15 - about to register with Runtime
** Agent agent15 is live
INFO:autogen_core.trace:** Agent agent15 is live
agent15: Received message
** Creator has created python code for agent agent16 - about to register with Runtime
** Agent agent16 is live
INFO:autogen_core.trace:** Agent agent16 is live
agent16: Received message
** Creator has created python code for agent agent4 - about to register with Runtime
** Agent agent4 is live
INFO:autogen_core.trace:** Agent agent4 is live
agent4: Received message
** Creator has created python code for agent agent5 - about to register with Runtime
** Agent agent5 is live
INFO:autogen_core.trace:** Agent agent5 is live
agent5: Received message
Selecting agent for refinement: agent7
agent7: Received message
Selecting agent for refinement: agent12
agent12: Received message
Selecting agent for refinement: agent16
agent16: Received message
Selecting agent for refinement: agent5
agent5: Received message
Selecting agent for refinement: agent18
agent18: Received message
Selecting agent for refinement: agent18
agent18: Received message
Selecting agent for refinement: agent2
agent2: Received message
Selecting agent for refinement: agent12
agent12: Received message
Selecting agent for refinement: agent10
agent10: Received message
Selecting agent for refinement: agent19
agent19: Received message
Selecting agent for refinement: agent11
agent11: Received message
Selecting agent for refinement: agent11
agent11: Received message
Selecting agent for refinement: agent12
agent12: Received message
Selecting agent for refinement: agent15
agent15: Received message
Selecting agent for refinement: agent11
agent11: Received message
Selecting agent for refinement: agent18
agent18: Received message
Selecting agent for refinement: agent3
agent3: Received message
Selecting agent for refinement: agent15
agent15: Received message
Selecting agent for refinement: agent10
agent10: Received message
Selecting agent for refinement: agent10
agent10: Received message
Selecting agent for refinement: agent18
agent18: Received message
Selecting agent for refinement: agent1
agent1: Received message
```

```sh
(agents_env) ➜  week5_autogen git:(main) ✗ tree -L 2                                                

.
├── 1_lab1_autogen_agentchat.ipynb
├── 2_lab2_autogen_agentchat.ipynb
├── 3_lab3_autogen_core copy.ipynb
├── 3_lab3_autogen_core.ipynb
├── 4_lab4_autogen_distributed copy.ipynb
├── 4_lab4_autogen_distributed.ipynb
├── 5_lab5_notes.ipynb
├── README.md
├── __pycache__
│   ├── agent.cpython-312.pyc
│   ├── agent1.cpython-312.pyc
│   ├── agent10.cpython-312.pyc
│   ├── agent11.cpython-312.pyc
│   ├── agent12.cpython-312.pyc
│   ├── agent13.cpython-312.pyc
│   ├── agent14.cpython-312.pyc
│   ├── agent15.cpython-312.pyc
│   ├── agent16.cpython-312.pyc
│   ├── agent17.cpython-312.pyc
│   ├── agent18.cpython-312.pyc
│   ├── agent19.cpython-312.pyc
│   ├── agent2.cpython-312.pyc
│   ├── agent20.cpython-312.pyc
│   ├── agent3.cpython-312.pyc
│   ├── agent4.cpython-312.pyc
│   ├── agent5.cpython-312.pyc
│   ├── agent6.cpython-312.pyc
│   ├── agent7.cpython-312.pyc
│   ├── agent8.cpython-312.pyc
│   ├── agent9.cpython-312.pyc
│   ├── creator.cpython-312.pyc
│   └── messages.cpython-312.pyc
├── agent.py
├── agent1.py
├── agent10.py
├── agent11.py
├── agent12.py
├── agent13.py
├── agent14.py
├── agent15.py
├── agent16.py
├── agent17.py
├── agent18.py
├── agent19.py
├── agent2.py
├── agent20.py
├── agent3.py
├── agent4.py
├── agent5.py
├── agent6.py
├── agent7.py
├── agent8.py
├── agent9.py
├── community_contributions
│   ├── 2_lab2_autogen_agentchat.ipynb
│   ├── 2_lab2_mcp_work_around
│   ├── community.ipynb
│   └── sandbox
├── creator.py
├── idea1.md
├── idea10.md
├── idea11.md
├── idea12.md
├── idea13.md
├── idea14.md
├── idea15.md
├── idea16.md
├── idea17.md
├── idea18.md
├── idea19.md
├── idea2.md
├── idea20.md
├── idea3.md
├── idea4.md
├── idea5.md
├── idea6.md
├── idea7.md
├── idea8.md
├── idea9.md
├── messages.py
├── sandbox
│   └── flights.md
├── tickets.db
└── world.py
```



**World.py: The Orchestration Script**

We're nearly there. I've just got one more thing to show you, which is world.py, which is quite short, which is the overall thing. World.py is not an agent. This is just a Python script. Everything else you've seen has been agents. This is just a Python script, but it uses async Python in quite an interesting way.

**How world.py Relates to creator.py**

Creator.py, you have to send it, it's an agent, and you have to send Creator a message, like agent1, agent2, agent3, and it will create them. So this is where it all comes together.

**Async Agent Creation: How It Works**

So there is this async function, this coroutine, createAndMessage, which takes a worker and a creator ID, and an i, which is going to be the agent number. We have 1, 2, 3, 4, 5. This how many agents up here is a pretty important one to look at. This is how many of these it's going to kick off at the same time.

**API Usage and Cost Considerations**

And the other thing that's unsafe about this, of course, is cost. This is going to hit the real APIs, and for me, it's cost like a couple of cents when I've done this for 20 agents, which is pretty cheap all things considered. And obviously, if you do it for like three agents, it costs nothing. But there's a risk there that we are allowing, in theory, the agent creator could change the model requirements. It could decide to use an expensive model instead of GPT-4 over any. And so that's just something to watch for. It's very unlikely, but it's not like it's not impossible.

**The Async Creation and Messaging Workflow**

Anyways, we've got 20 agents here, so we've got this async method, creator, and endMessage. It takes this worker and it sends a message to the agent number whatever dot dot pi, and then it writes the result to idea whatever dot marker. And that's all this thing here does.

**The Main Coroutine: Parallel Launching**

Okay, so now we go to our main async method and our main async care function, our covert team. So it creates a host, a gRPC worker agent runtime host, as we did in the notebook before. It starts the host, it creates a worker, and it starts the worker. It then registers the creator. So it registers the creator from creator dot pi that was imported here. So that will launch a creator. And this is its ID, creator default.

**Why Use Coroutines and asyncio.gather?**

Then this is the async thing. So I don't want to have a loop and call this creating worker after worker, sorry, creating agent after agent. Because then it will be serial. It will happen one after another. If I do await and then I call this coroutine, then we'll be waiting there. It will create one file, it will send it a message, back will come the answer, maybe it will send it somewhere else, and then it will go on to the next. And we'll be sitting here waiting, and it won't be exciting. But what you can do with coroutines, as you hopefully remember from when we briefly talked about this, is that you can get a whole stash of coroutines together. So I haven't got the word await here, which you need if this is actually going to run. I'm just gathering a whole list of coroutines. And then I await them using asyncIO dot gather.

**AsyncIO and Event Loop**

And asyncIO dot gather runs them all in parallel. And as hopefully you remember, it's not like multithreading. They're not going to be actually running on different threads, like with the CPU chopping between them. But rather, they're running in the event loop. That's how asyncIO Python works. They run an event loop so that every time it's waiting on OpenAI, which means it's waiting on a network connection, another one can be running. And that means they all get to run at the same time. As long as I stay within my rate limits with OpenAI, we'll be able to run plenty in parallel.

**Finalizing and Running the Orchestration**

And so it does all of that, and then at the end it stops, and any exceptions, it will print them. And that's it. This is, by the way, how you run asyncPython from the command line or from a Python module. Your main file needs to do this, which brings, which initiates an asyncIO event loop and then runs your async method, your coroutine. So that is how it hangs together.

**Summary and Recap**

And I do believe this is it. I don't think I've got anything else to show you. I think you've seen everything. You've seen agent, which is the prototype, the thing that gets cloned. Creator, which does the cloning agent by agent. It's called one by one. And world.py is this. It's not an agent. It's just Python code, which will orchestrate, which will launch the whole process running. Okay.


---



### **Agent 16: FinBuddy – Financial Services Idea**


**Refinement and Review**

So I absolutely love the vision behind FinBuddy.
It's a powerful concept with potential to make significant impact on financial literacy and accessibility for underserved communities.
Let's refine it and elevate the idea even further.
Refined concept, so this is great.
It's an idea for applying agentic AI to financial services that's been refined by another agent.

**FinBuddy: The Concept**

**FinBuddy:** AI-powered financial companion for empowerment and inclusivity.
So it's basically an education platform for people who are underserved in financial services.

* Contextual financial advisor
* Localized community support hubs
* Micro-investment pools

  * Users can join collaborative investment pools, collectively decide on small-scale investments.
    That is cool.

**Features and Gamification**

* Engagement-driven gamification with real-world rewards
* Partner with local businesses
  Interesting. Very interesting.

**Personal Comment and Closing Thoughts**

Well, I'm not going to read it. You might not hear from me again.
So anyway, the goal of this is to give you a live agentic platform where agents are created and agents collaborate and interact in an autonomous way.
In such an autonomous way that they were even created by another agent.

**Reflection on the Project**

And I hope you find this as satisfying as I do.
I hope it was worth it. I hope it was worth the half-an-hour investment to get here.
And do take a look through the code. If nothing else, as I say, it's definitely got some interesting ideas built in there about how to use things like AutoGen for this kind of messaging between agents.
And if nothing else, if you give it a run, if you feel bold, you will find a bunch of ideas.
And maybe you'll be off on your yacht before too long.
All right. I'll see you for the wrap-up.

---


Well, thank you for indulging me for the last half-an-hour of going through that project.
I do hope that you at least found it somewhat interesting, educational, and opened your eyes a bit into some of what's possible at the frontier of agentic AI, which is very much where AutoGen is.
It's very much an experimental and futuristic platform, as we've seen ourselves firsthand.




---

### Let's take a look at Agent 16

Let's take a look at Agent 16 and see the financial services ideas.
Let's open a preview and let's get rid of this.
Oh, and the fact that we've got this, I absolutely love the vision behind FinBuddy.
This thing at the top here tells us that this was sent to another agent for feedback.
At least one agent added feedback to this. So this went through a couple of people.


```sh
(agents_env) ➜  week5_autogen git:(main) ✗ cat idea16.md 
```

>Your idea for **"Quest for Knowledge"** is absolutely fantastic! It beautifully intertwines gaming, >education, and immersive experiences. Let’s refine and enhance it to maximize its potential and >appeal:
>
>### Refined Concept: **"Quest for Knowledge"**
>
>Imagine a multi-dimensional platform that transforms learning into an exhilarating adventure, allowing users to not only engage with educational content but also to be active participants in their own learning journeys.
>
>### Enhanced Key Features:
>
>1. **Dynamic Interactive Storytelling:**
>   - **Branching Narratives:** Develop narratives with multiple outcomes based on players' choices, allowing them to experience different perspectives (e.g., a Roman citizen vs. a gladiator). This encourages critical thinking and empathy.
>   - **Epic Lore:** Each quest could be embedded with rich lore, characters, and side quests that deepen the educational content while keeping the gaming experience engaging.
>
>2. **Seamless Augmented Reality Integration:**
>   - **Geo-Tagging:** Users could unlock AR challenges tied to their geographical location. For instance, standing in a park might prompt a challenge related to the local ecosystem, or visiting a museum could reveal interactive AR exhibits.
>   - **AR Mini-Games:** Introduce mini-games that blend with real-world objects. For instance, a virtual fossil excavation could teach paleontology concepts, allowing learners to dig up "fossils" in their backyard.
>
>3. **Collaborative Community Challenges:**
>   - **Global Quests:** Organize global events where players from different parts of the world collaborate on solving a complex problem or completing a challenge, promoting cultural exchange and teamwork.
>   - **Leaderboard and Achievements:** Implement a leaderboard system for community challenges to encourage friendly competition while celebrating individual and team achievements.
>
>4. **Expert Mentorship and Networking:**
>   - **Virtual Meet-and-Greets:** Create opportunities for players to interact with experts in a virtual amphitheater, where they can ask questions and participate in moderated discussions.
>   - **Peer Mentorship:** Introduce a system where experienced players can mentor newcomers, fostering a supportive learning environment and building community.
>
>5. **Personalized Learning Experience:**
>   - **Adaptive Learning Paths:** Use AI to analyze players’ strengths and weaknesses, offering tailored quests that align with their learning objectives. This ensures every player has a unique and relevant learning journey.
>   - **Progressive Skill Trees:** Incorporate skill trees that allow players to specialize in certain areas (e.g., history, science, literature) as they advance, providing a sense of growth and mastery.
>
>6. **Immersive Soundscapes and Visuals:**
>   - **360-Degree Environments:** Create stunning VR environments with rich, immersive soundscapes that enhance the learning experience. For instance, the sound of bustling markets in ancient Rome or the calls of ocean life during marine quests.
>   - **Interactive Artifacts:** Introduce objects players can collect or interact with during their quests that provide additional information or unlock hidden challenges.
>
>### Final Touch:
>To truly make **"Quest for Knowledge"** a revolution in education, consider incorporating a **social impact element** where players can contribute to real-world educational initiatives. For example, completing certain quests could generate donations to schools or educational programs in underserved communities.
>
>This platform not only captivates users with its engaging approach to learning but also cultivates a sense of community, collaboration, and social responsibility. 
>
>What do you think about these enhancements? Would they elevate your concept to the next level?



### **End-of-Week Challenge: Experiment, Research, and Go Meta**

So the challenge for you, the end-of-week challenge, is an optional one, because this week has been experimental and a researchy week.
But, should you wish, invest some more time in this idea, see if you can work with me on making it a bit more robust, and something that's safer for people to run, perhaps, by dockerizing it.



**Taking It Further: Self-Creating Creators**

But in particular, something which I think would be fascinating, would be to have it so that the creator is not only able to write new agents, but it's also able to write a new version of itself.
It can create a new creator, modeling it off its own template, of its own file, which it can also read.
It might be interesting to turn it into a tool, rather than just having it be there in the Python code.
Have it be able to run a tool that reads in its own code, and it can then rewrite itself, making it a replica.
Something which itself is able to create new agents, and perhaps it could change its own logic slightly in some interesting way.

So I think that would be fascinating. Super meta.
It's something that creates creators.
And I mean, if it creates itself, then in theory it can create creators of creators.
So, yeah, mind-blows. Interesting, interesting project.

**Experimenting with Environments and Messaging**

A great way, importantly, to be experimenting with interactions between agents in this kind of idea of an environment where the messaging, the communication and creation of agents, is a separate concern from the implementation of the agents, which is, of course, the fundamental point of Uta Janko.

**Looking Ahead: Conclusion and Next Steps**

All right, and with that, finally, I'll stop yammering away about this.
The agent creator is done, and we are on to week six.
The fantastic, the exciting conclusion, and it is going to be a legendary week.

I can't wait to show you everything that MCP is about and can offer, and I can't wait to return to OpenAI Agents SDK, still my favorite, even after enjoying all of the others, enjoying CRU and Landgraf a lot, and being generally entertained by Autogen and by its board pickings.
But OpenAI Agents SDK is next, with MCP.
I'll see you then.
Thank you.

